## Doing a bit of Data Analysis 😎

This won't stay in the project but it's easier to do this in a notebook while developing

In [62]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

In [3]:
CSV_FILE = "data/full/cineasta_full.csv"

In [4]:
cineData = pd.read_csv(CSV_FILE, parse_dates=['Date'])
cineData = cineData.drop(columns=["Unnamed: 0"])

In [13]:
cineData.dtypes

OG_Title                object
TMDB_Title              object
Release_Date            object
Rating                 float64
Genre_IDs               object
MovieAge                object
Semester                object
Weekday                 object
Date            datetime64[ns]
Attendance               int64
dtype: object

In [5]:
cineData.head()

,OG_Title,TMDB_Title,Release_Date,Rating,Genre_IDs,MovieAge,Semester,Weekday,Date,Attendance
0,Kingsman 2,Kingsman: The Golden Circle,2017-09-20,6.969,"['Action', 'Adventure', 'Comedy']",202 days,Sommersemester,Tuesday,2018-04-10,87
1,Ferris Bueller's Day off,Ferris Bueller's Day Off,1986-06-11,7.605,['Comedy'],11627 days,Sommersemester,Wednesday,2018-04-11,16
2,Detroit,Detroit,2017-07-28,7.300,"['Crime', 'Drama', 'Thriller', 'History']",264 days,Sommersemester,Wednesday,2018-04-18,9
3,It,It,2017-09-06,7.243,"['Horror', 'Fantasy']",230 days,Sommersemester,Tuesday,2018-04-24,102
4,The Square,The Square,2017-08-25,6.765,['Drama'],243 days,Sommersemester,Wednesday,2018-04-25,30


In [6]:
#after covid
a_cov = cineData[cineData['Date'] > pd.to_datetime('2020-3-1')]
#before covid
b_cov = cineData[cineData['Date'] < pd.to_datetime('2020-3-1')]

In [7]:
print(f"Median before Covid: {b_cov['Attendance'].median()}")
print(f"Median after Covid: {a_cov['Attendance'].median()}")

Median before Covid: 33.0
Median after Covid: 9.0


In [10]:
semDiff_a = a_cov.groupby(['Semester']).Attendance.mean()
semDiff_b = b_cov.groupby(['Semester']).Attendance.mean()

In [ ]:
plt.bar(semDiff_b.index, semDiff_b.values, color = 'orange')
plt.bar(semDiff_a.index, semDiff_a.values, color='blue')
plt.xlabel('Semester')
plt.ylabel('Attendance')
plt.legend(['Before Covid', 'After Covid'])
plt.title("Median of Attendance")